## 0.0. Install/Import Dependencies

In [ ]:
import pandas as pd
import numpy as np

,league,season,rank,team,matches,wins,draws,losses,goals,goalDiff,points
0,Serie A,2018/2019,1,Juventus,38,28,6,4,70:30,40,90
1,Serie A,2018/2019,2,SSC Napoli,38,24,7,7,74:36,38,79
2,Serie A,2018/2019,3,Atalanta,38,20,9,9,77:46,31,69
3,Serie A,2018/2019,4,Inter,38,20,9,9,57:33,24,69
4,Serie A,2018/2019,5,AC Milan,38,19,11,8,55:36,19,68
...,...,...,...,...,...,...,...,...,...,...,...
2813,Primera División,2016/2017,16,Deportivo La Coruña,38,8,12,18,43:61,-18,36
2814,Primera División,2016/2017,17,CD Leganés,38,8,11,19,36:55,-19,35
2815,Primera División,2016/2017,18,Sporting Gijón,38,7,10,21,42:72,-30,31
2816,Primera División,2016/2017,19,CA Osasuna,38,4,10,24,40:94,-54,22


## 0.1. Reading and Cleaning Scraped Data

In [ ]:
df = pd.read_csv('top5leagueSeasons.csv')
df

In [ ]:
df.dtypes

In [ ]:
df['goalDiffpg'] = np.around(df.goalDiff.values / df.matches.values, decimals=2)
df['pointstype'] = df.points.apply(lambda x: 2 if ':' in str(x) else 3)
df['points'] = df.points.apply(lambda x: int(str(x).split(':')[0]))
df['ppg'] = df.apply(lambda x: np.around((x.points / x.matches) / x.pointstype, decimals=2), axis=1)
df['goalsfor'] = df.goals.apply(lambda x: int(str(x).split(':')[0]))
df['goalsagainst'] = df.goals.apply(lambda x: int(str(x).split(':')[1]))
df

## 1. Introduction

As more money pours into European football in the form of foreign investment, many fans are left concerned that the level of competition is slowly dwindling and all the power lies in the hands of the biggest clubs. While that may be true in many regards, the main aspect of this dynamic that I wanted to look into is the level of competition within each Top 5 leagues (England, France, Germany, Spain, and Italy) since the first season of the Premier League (1992/1993). The founding of England's modern top flight format is seen as one of the first attempts in world football to consolidate power and financial resources, and since then other governing bodies have followed suit. Through looking at key parity metrics over time, I aim to expose trends in how intra-league competitiveness has evolved from 1992 to now.

## 2. Metric Explanations and Calculations

### 2.1. Difference in League Position from Previous Season

To capture variation in league position, I calculated the difference between a team's final position and the position they finished in the prior year. The relegation system introduced complexity to this problem as the vast majority of teams were not in the top division for the entire period. Therefore, they would have no ranking to use for the previous season to calculate the difference. To fill there null values I took the following approach:

First I forward filled to use the league standing from their last season in the top flight -- robust against successful teams that were relegated for specific violations (Juventus-Italy 06/07)
However, this did not account for teams whose did not appear in the top flight prior to that season
To address these teams, I calculated the difference between their final standing and the relegation boundary of each league.

In [ ]:
# map to store relegation zone boundary for each league
relegation_zone = {
    'Premier League': 18,
    'Serie A': 18,
    'Bundesliga': 16,
    'Ligue 1': 18,
    'Primera División': 18,
}

# if team is present for all 29 seasons then they haven't been relegated
def relegation_check(team_name):
    if df[df.team == team_name].shape[0] == 29:
        return False
    else:
        return True

# for seasons spent in lower division, generates df accounting for those w/ rank np.nan
def fill_missing_seasons(team_name):
    missingdf = df[['season', 'league', 'rank', 'team']][df.team == team_name]
    league = missingdf.iat[0,1]
    team = missingdf.iat[0,3]
    g_seasons = set(missingdf.season.unique())
    seasons = set(df.season.unique())
    seasons_rele = seasons.difference(g_seasons) # seasons spent in lower div
    relegated_seasons = [{'season': season, 'league': league, 'rank': np.nan, 'team': team}
                        for season in seasons_rele]
    relegateddf = pd.DataFrame(relegated_seasons)
    all_seasonsdf = pd.concat([missingdf, relegateddf])
    return all_seasonsdf.sort_values('season')

def rank_na_fill(team_name):
    if relegation_check(team_name): # team has been relegated
        teamdf = fill_missing_seasons(team_name)
        teamdf['lower_div'] = teamdf['rank'].isna()
        teamdf['relegated'] = teamdf['rank'].isna().shift(-1) # if they spent
        teamdf['rankfill'] = teamdf['rank'].ffill().fillna(relegation_zone[teamdf.iat[0,1]])
        teamdf['rankdiff'] = teamdf['rankfill'].diff().abs()
    else: # team has not so no missing seassons
        teamdf = df[['season', 'league', 'rank', 'team']][df.team == team_name].sort_values('season')
        teamdf['lower_div'] = False
        teamdf['relegated'] = teamdf.lower_div.shift(-1)
        teamdf['rankdiff'] = teamdf['rank'].diff().abs()

    # both conditionals output teamdf with same columns
    return teamdf[teamdf.lower_div == False][['season', 'league', 'team', 'rankdiff']]

relegationdfs = [rank_na_fill(team) for team in df.team.unique()]
relegationdf = pd.concat(relegationdfs)
relegationdf

,season,league,team,rankdiff
2119,1992/1993,Serie A,Juventus,NaN
1835,1993/1994,Serie A,Juventus,2.0
1546,1994/1995,Serie A,Juventus,1.0
263,1995/1996,Serie A,Juventus,1.0
704,1996/1997,Serie A,Juventus,1.0
...,...,...,...,...
2599,2014/2015,Primera División,Córdoba CF,2.0
2609,2017/2018,Primera División,Girona FC,8.0
2677,2018/2019,Primera División,Girona FC,8.0
2738,2010/2011,Premier League,Blackpool FC,1.0


In [ ]:
# merging relegation df to main df to consolidate data export
masterdf = df.merge(relegationdf, on=['season', 'league', 'team'])
masterdf

,league,season,rank,team,matches,wins,draws,losses,goals,goalDiff,points,goalDiffpg,pointstype,ppg,goalsfor,goalsagainst,rankdiff
0,Serie A,2018/2019,1,Juventus,38,28,6,4,70:30,40,90,1.05,3,0.79,70,30,0.0
1,Serie A,2018/2019,2,SSC Napoli,38,24,7,7,74:36,38,79,1.00,3,0.69,74,36,0.0
2,Serie A,2018/2019,3,Atalanta,38,20,9,9,77:46,31,69,0.82,3,0.61,77,46,4.0
3,Serie A,2018/2019,4,Inter,38,20,9,9,57:33,24,69,0.63,3,0.61,57,33,0.0
4,Serie A,2018/2019,5,AC Milan,38,19,11,8,55:36,19,68,0.50,3,0.60,55,36,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2813,Primera División,2016/2017,16,Deportivo La Coruña,38,8,12,18,43:61,-18,36,-0.47,3,0.32,43,61,1.0
2814,Primera División,2016/2017,17,CD Leganés,38,8,11,19,36:55,-19,35,-0.50,3,0.31,36,55,1.0
2815,Primera División,2016/2017,18,Sporting Gijón,38,7,10,21,42:72,-30,31,-0.79,3,0.27,42,72,1.0
2816,Primera División,2016/2017,19,CA Osasuna,38,4,10,24,40:94,-54,22,-1.42,3,0.19,40,94,1.0


### 2.2. 4-year Rolling Window Metrics



Another measure of balance in league playing field is how many unique teams win the championship and occupy the top 4 places within a 4-year period. The more competitive a league, the more unique teams that would occupy these places in the league standings. As the Top 4 represents which teams qualify for the champions league, it is often seen as a benchmark for season preformance, so it is important to include with league winner distinct count.

Since pandas does not allow for rolling or groupby aggregates to manipulate string data, I grouped by season and league to generate an array of the teams in the top 4, or in the case of the champion metric, a single string of the team that won. Then I wrote a custom function to get the number of unique teams within the 4 year window.

In [ ]:
from operator import add

seasontop4s = df[df['rank'] <= 4].sort_values(['season', 'league']) # filter teams to just ones that finished in top 4
teamsintop4 = seasontop4s[['season', 'league', 'team']].groupby(['season', 'league']).agg({'team': pd.Series.unique}).reset_index() # array of unique teams
teamsintop4['team'] = teamsintop4.team.apply(lambda x: x.tolist())

def unique_teams(series):
    rolling_count = []
    for i in range(0, 29):
        if i <= 2:
            rolling_count.append(np.nan)
        else:
            unique_teams = set(reduce(add, series[i-3:i+1])) # i-3:i+3 defines window
            rolling_count.append(len(unique_teams))
    return pd.Series(rolling_count)

def get_top4_window(league_name):
    leaguedf = teamsintop4[teamsintop4.league == league_name].sort_values('season').reset_index(drop=True)
    leaguedf['uniquetop4'] = unique_teams(leaguedf.team.values)
    return leaguedf.drop(columns = ['team'])

top4dfs = [get_top4_window(league) for league in df.league.unique()]
top4df = pd.concat(top4dfs)
top4df

,season,league,uniquetop4
0,1992/1993,Serie A,NaN
1,1993/1994,Serie A,NaN
2,1994/1995,Serie A,NaN
3,1995/1996,Serie A,7.0
4,1996/1997,Serie A,7.0
...,...,...,...
24,2016/2017,Premier League,7.0
25,2017/2018,Premier League,7.0
26,2018/2019,Premier League,7.0
27,2019/2020,Premier League,5.0


In [ ]:
league_winners = df[df['rank'] == 1].sort_values(['season', 'league'])
team_winners = league_winners[['season', 'league', 'team']].groupby(['season', 'league']).agg({'team': pd.Series.unique}).reset_index()

def unique_teams(series):
    rolling_count = []
    for i in range(0, 29):
        if i <= 2:
            rolling_count.append(np.nan)
        else:
            unique_teams = set(series[i-3:i+1])
            rolling_count.append(len(unique_teams))
    return pd.Series(rolling_count)

def get_winner_window(league_name):
    leaguedf = team_winners[team_winners.league == league_name].sort_values('season').reset_index(drop=True)
    leaguedf['uniquechamps'] = unique_teams(leaguedf.team.values)
    return leaguedf.drop(columns = ['team'])

winnerdfs = [get_winner_window(league) for league in df.league.unique()]
winnerdf = pd.concat(winnerdfs)
winnerdf

,season,league,uniquechamps
0,1992/1993,Serie A,NaN
1,1993/1994,Serie A,NaN
2,1994/1995,Serie A,NaN
3,1995/1996,Serie A,2.0
4,1996/1997,Serie A,2.0
...,...,...,...
24,2016/2017,Premier League,3.0
25,2017/2018,Premier League,3.0
26,2018/2019,Premier League,3.0
27,2019/2020,Premier League,3.0


In [ ]:
windowaggdf = top4df.merge(winnerdf, on=['season', 'league'])
windowaggdf

,season,league,uniquetop4,uniquechamps
0,1992/1993,Serie A,NaN,NaN
1,1993/1994,Serie A,NaN,NaN
2,1994/1995,Serie A,NaN,NaN
3,1995/1996,Serie A,7.0,2.0
4,1996/1997,Serie A,7.0,2.0
...,...,...,...,...
140,2016/2017,Premier League,7.0,3.0
141,2017/2018,Premier League,7.0,3.0
142,2018/2019,Premier League,7.0,3.0
143,2019/2020,Premier League,5.0,3.0


### 2.3. Point difference between 1st and 2nd and 1st and 3rd

To generate a metric for the titleraces specifically, I calculated the difference in points between 1st and 2nd league positions, and 1st and 3rd. The larger the gap between these teams, the less competitive the given season was.

Here, I was actually able to use the pd.groupby().agg() method given the numerical nature of the rank column.

In [ ]:
# point margin of title race
titleraces = df[df['rank'] <= 3]

def first_to_second(series):
    first_to_second = series.iloc[0] - series.iloc[1] 
    return first_to_second

def first_to_third(series):
    first_to_third = series.iloc[0] - series.iloc[2]
    return first_to_third

season_point_margins = titleraces.groupby(['season', 'league'])['points'].agg([first_to_second, first_to_third]).reset_index()
season_point_margins

,season,league,first_to_second,first_to_third
0,1992/1993,Bundesliga,1,6
1,1992/1993,Ligue 1,2,2
2,1992/1993,Premier League,10,12
3,1992/1993,Primera División,1,4
4,1992/1993,Serie A,4,9
...,...,...,...,...
140,2020/2021,Bundesliga,13,14
141,2020/2021,Ligue 1,1,5
142,2020/2021,Premier League,12,17
143,2020/2021,Primera División,2,7


### 2.4. Consolidating League/Season Aggregates to one DataFrame

In [ ]:
leagueseasondf = windowaggdf.merge(season_point_margins, on=['season', 'league'])
leagueseasondf

## 3. Exporting DataFames as csv files for Tableu Dashboard

With the metric calculated and dataframes merged, they can be exported as csvs and imported into Tableu Public to drive data visualizations inside an interactive dashboard.

In [ ]:
# Export datafames to csvs
from google.colab import files

masterdf.to_csv('seasontables.csv')
leagueseasondf.to_csv('seasonaggs.csv')